# Bespoke-Stratos-17k 数据集 Pattern 提取验证

本脚本用于验证 Bespoke-Stratos-17k 数据库的答案能否被现有的 pattern 提取脚本识别出来。

## 验证目标

1. 验证数据集中的答案格式是否能够被 `match_format` 正则表达式识别
2. 验证数据集中的数值答案是否能够被 `match_numbers` 正则表达式提取
3. 统计识别成功率
4. 展示成功和失败的示例

In [26]:
# ============================================================================
# 安装必要的依赖
# ============================================================================
import importlib.util

def check_package(name):
    """检查包是否已安装"""
    return importlib.util.find_spec(name) is not None

if not check_package('datasets'):
    print("正在安装 datasets...")
    %pip install -q datasets
    print("安装完成！")

if not check_package('tqdm'):
    print("正在安装 tqdm...")
    %pip install -q tqdm
    print("安装完成！")

print("所有依赖已就绪！")

所有依赖已就绪！


In [27]:
# ============================================================================
# 导入必要的库
# ============================================================================
import re
from datasets import load_dataset
from tqdm import tqdm

print("库导入完成！")

库导入完成！


In [28]:
# ============================================================================
# 定义 Pattern 提取脚本（与训练脚本保持一致）
# ============================================================================

# 定义特殊标记
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"

# 正则表达式：验证输出格式
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{re.escape(reasoning_start)}.+?{re.escape(reasoning_end)}.*?"
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)

# 正则表达式：提取答案中的数字
match_numbers = re.compile(
    rf"{re.escape(solution_start)}.*?([\d\.]{{1,}})",
    flags=re.MULTILINE | re.DOTALL
)

print("Pattern 定义完成！")
print(f"Format pattern: {match_format.pattern[:100]}...")
print(f"Numbers pattern: {match_numbers.pattern}")

Pattern 定义完成！
Format pattern: ^[\s]{0,}<reasoning>.+?</reasoning>.*?<answer>(.+?)</answer>[\s]{0,}$...
Numbers pattern: <answer>.*?([\d\.]{1,})


In [29]:
# ============================================================================
# 加载并转换 Bespoke-Stratos-17k 数据集
# ============================================================================

def load_and_convert_bespoke_stratos_dataset(num_examples=100):
    """
    加载 Bespoke-Stratos-17k 数据集并转换为标准格式
    
    Args:
        num_examples: 要加载的样本数量（None = 全部）
    
    Returns:
        List of dicts with 'original', 'converted', 'user_msg' keys
    """
    print("正在加载 Bespoke-Stratos-17k 数据集...")
    
    # 从 HuggingFace 加载数据集
    dataset = load_dataset("bespokelabs/Bespoke-Stratos-17k", split="train")
    
    converted_data = []
    for i, example in enumerate(tqdm(dataset, desc="处理样本")):
        if num_examples and i >= num_examples:
            break
        
        # 提取对话
        conversations = example.get("conversations", [])
        if len(conversations) < 2:
            continue
        
        # 获取用户消息和助手回复
        user_msg = None
        assistant_msg = None
        
        for conv in conversations:
            role = conv.get("from", conv.get("role", ""))
            content = conv.get("value", conv.get("content", ""))
            
            if role in ["human", "user"]:
                user_msg = content
            elif role in ["gpt", "assistant"]:
                assistant_msg = content
        
        if not user_msg or not assistant_msg:
            continue
        
        # 保存原始消息
        original_completion = assistant_msg
        
        # 转换各种思考标签为标准 <reasoning>...</reasoning> 格式
        completion = assistant_msg
        
        # 处理 Bespoke-Stratos 格式
        completion = completion.replace("<|begin_of_thought|>", reasoning_start)
        completion = completion.replace("<|end_of_thought|>", reasoning_end)
        completion = completion.replace("<|begin_of_solution|>", solution_start)
        completion = completion.replace("<|end_of_solution|>", solution_end)
        
        # 处理其他常见格式
        completion = completion.replace("<think>", reasoning_start)
        completion = completion.replace("</think>", reasoning_end)
        completion = re.sub(r"<think>", reasoning_start, completion, flags=re.IGNORECASE)
        completion = re.sub(r"</think>", reasoning_end, completion, flags=re.IGNORECASE)
        completion = re.sub(r"<thinking>", reasoning_start, completion, flags=re.IGNORECASE)
        completion = re.sub(r"</thinking>", reasoning_end, completion, flags=re.IGNORECASE)
        
        converted_data.append({
            "index": i,
            "original": original_completion,
            "converted": completion,
            "user_msg": user_msg,
        })
    
    print(f"加载并转换了 {len(converted_data)} 个样本")
    return converted_data

# 加载数据（可以调整数量以加快测试）
NUM_SAMPLES = 500  # 加载500个样本进行验证
data = load_and_convert_bespoke_stratos_dataset(num_examples=NUM_SAMPLES)

正在加载 Bespoke-Stratos-17k 数据集...


处理样本:   3%|▎         | 500/16710 [00:00<00:01, 9988.01it/s]

加载并转换了 500 个样本


In [40]:
# ============================================================================
# 验证 Pattern 识别能力
# ============================================================================

def validate_patterns(data):
    """
    验证转换后的数据是否能够被 pattern 识别
    
    Returns:
        dict with statistics and examples
    """
    format_matches = 0
    number_extractions = 0
    total = len(data)
    
    success_examples = []
    format_fail_examples = []
    number_fail_examples = []
    
    for item in tqdm(data, desc="验证 Pattern"):
        converted = item["converted"]
        
        # 1. 检查格式匹配
        format_match = match_format.search(converted)
        if format_match:
            format_matches += 1
        else:
            format_fail_examples.append(item)
            continue
        
        # 2. 检查数字提取
        number_match = match_numbers.search(converted)
        if number_match:
            number_extractions += 1
            extracted_number = number_match.group(1)
            success_examples.append({
                "index": item["index"],
                "converted": converted,  
                "extracted_number": extracted_number
            })
        else:
            number_fail_examples.append(item)
    
    return {
        "total": total,
        "format_matches": format_matches,
        "number_extractions": number_extractions,
        "format_match_rate": format_matches / total * 100,
        "number_extraction_rate": number_extractions / total * 100,
        "success_examples": success_examples[:5],  # 保存5个成功示例
        "format_fail_examples": format_fail_examples[:5],  # 保存5个格式失败示例
        "number_fail_examples": number_fail_examples[:5],  # 保存5个数字提取失败示例
    }

# 运行验证
results = validate_patterns(data)

验证 Pattern: 100%|██████████| 500/500 [00:00<00:00, 9810.27it/s]


In [41]:
# ============================================================================
# 显示验证结果统计
# ============================================================================

print("=" * 80)
print("验证结果统计")
print("=" * 80)
print(f"总样本数: {results['total']}")
print(f"格式匹配成功: {results['format_matches']} ({results['format_match_rate']:.2f}%)")
print(f"数字提取成功: {results['number_extractions']} ({results['number_extraction_rate']:.2f}%)")
print("=" * 80)

if results['format_match_rate'] >= 90:
    print("✅ 格式匹配率优秀！")
elif results['format_match_rate'] >= 70:
    print("⚠️  格式匹配率良好，但有改进空间")
else:
    print("❌ 格式匹配率较低，需要检查数据转换逻辑")

if results['number_extraction_rate'] >= 70:
    print("✅ 数字提取率良好！")
elif results['number_extraction_rate'] >= 50:
    print("⚠️  数字提取率一般，部分答案可能不是纯数字")
else:
    print("❌ 数字提取率较低，可能需要调整提取逻辑")

验证结果统计
总样本数: 500
格式匹配成功: 500 (100.00%)
数字提取成功: 500 (100.00%)
✅ 格式匹配率优秀！
✅ 数字提取率良好！


In [42]:
# ============================================================================
# 显示成功示例
# ============================================================================

print("=" * 80)
print("成功识别示例（前5个）")
print("=" * 80)

for i, example in enumerate(results['success_examples'], 1):
    print(f"\n示例 {i} (索引: {example['index']}):")
    print("-" * 80)
    print(f"提取的数字: {example['extracted_number']}")
    print(f"转换后的内容（前500字符）:\n{example['converted']}")
    print()

成功识别示例（前5个）

示例 1 (索引: 0):
--------------------------------------------------------------------------------
提取的数字: 1
转换后的内容（前500字符）:
<reasoning>

Okay, let me try to figure out this problem. So, we have this operation defined as a⊗b = a²/b. And we need to compute [(1⊗2)⊗3] - [1⊗(2⊗3)]. Then choose the correct answer from the options given. Alright, let's break it down step by step.

First, I need to remember that the operation ⊗ is not associative, right? Because the problem is asking for the difference between two different groupings: (1⊗2)⊗3 and 1⊗(2⊗3). So, the order in which we perform the operations matters here. That's probably why there's a subtraction between them.

Let me start by computing each part separately. Let's tackle the first part: (1⊗2)⊗3.

Starting with the innermost operation, which is 1⊗2. According to the definition, a⊗b = a²/b. So here, a is 1 and b is 2. Plugging those in: 1² / 2 = 1/2. So, 1⊗2 equals 1/2.

Now, we take that result and perform the next operatio

In [33]:
# ============================================================================
# 显示格式匹配失败的示例
# ============================================================================

if results['format_fail_examples']:
    print("=" * 80)
    print("格式匹配失败的示例（前5个）")
    print("=" * 80)
    
    for i, example in enumerate(results['format_fail_examples'], 1):
        print(f"\n示例 {i} (索引: {example['index']}):")
        print("-" * 80)
        print("原始内容:")
        print(example['original'][:300])
        print("\n转换后的内容:")
        print(example['converted'][:500])
        print("\n失败原因分析:")
        
        has_reasoning_start = reasoning_start in example['converted']
        has_reasoning_end = reasoning_end in example['converted']
        has_answer_start = solution_start in example['converted']
        has_answer_end = solution_end in example['converted']
        
        print(f"  - 包含 <reasoning>: {has_reasoning_start}")
        print(f"  - 包含 </reasoning>: {has_reasoning_end}")
        print(f"  - 包含 <answer>: {has_answer_start}")
        print(f"  - 包含 </answer>: {has_answer_end}")
        print()
else:
    print("✅ 没有格式匹配失败的示例！")

✅ 没有格式匹配失败的示例！


In [34]:
# ============================================================================
# 显示数字提取失败的示例
# ============================================================================

if results['number_fail_examples']:
    print("=" * 80)
    print("数字提取失败的示例（前5个）")
    print("=" * 80)
    
    for i, example in enumerate(results['number_fail_examples'], 1):
        print(f"\n示例 {i} (索引: {example['index']}):")
        print("-" * 80)
        
        # 尝试提取 answer 标签内的内容
        answer_match = re.search(
            rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}",
            example['converted'],
            flags=re.DOTALL
        )
        
        if answer_match:
            answer_content = answer_match.group(1).strip()
            print(f"<answer> 标签内的内容: {answer_content[:200]}")
            
            # 检查是否包含数字
            has_digits = bool(re.search(r'\d', answer_content))
            print(f"是否包含数字: {has_digits}")
            
            if not has_digits:
                print("⚠️  答案内容不包含数字，可能是文本答案")
            else:
                print("⚠️  答案包含数字但提取失败，可能需要调整提取模式")
        else:
            print("无法找到 <answer> 标签")
        
        print(f"\n转换后的内容（前500字符）:\n{example['converted'][:500]}")
        print()
else:
    print("✅ 所有格式匹配成功的样本都能提取数字！")

✅ 所有格式匹配成功的样本都能提取数字！


In [35]:
# ============================================================================
# 详细分析：检查 answer 标签内的内容类型
# ============================================================================

print("=" * 80)
print("答案内容类型分析")
print("=" * 80)

answer_types = {
    "纯数字": 0,
    "包含数字的文本": 0,
    "纯文本": 0,
    "无answer标签": 0
}

answer_pattern = re.compile(
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}",
    flags=re.DOTALL
)

for item in data:
    converted = item["converted"]
    answer_match = answer_pattern.search(converted)
    
    if not answer_match:
        answer_types["无answer标签"] += 1
        continue
    
    answer_content = answer_match.group(1).strip()
    
    # 检查是否只有数字（可能包含小数点）
    if re.match(r'^[\d\.]+$', answer_content):
        answer_types["纯数字"] += 1
    elif re.search(r'\d', answer_content):
        answer_types["包含数字的文本"] += 1
    else:
        answer_types["纯文本"] += 1

total = len(data)
print(f"总样本数: {total}")
for answer_type, count in answer_types.items():
    percentage = count / total * 100
    print(f"{answer_type}: {count} ({percentage:.2f}%)")

print("\n说明:")
print("- 纯数字: 答案只包含数字，match_numbers 应该能够提取")
print("- 包含数字的文本: 答案包含数字但也包含其他文本，可能需要调整提取逻辑")
print("- 纯文本: 答案不包含数字，这类问题可能不是数学问题")
print("- 无answer标签: 转换后仍然没有 answer 标签")

答案内容类型分析
总样本数: 500
纯数字: 0 (0.00%)
包含数字的文本: 496 (99.20%)
纯文本: 4 (0.80%)
无answer标签: 0 (0.00%)

说明:
- 纯数字: 答案只包含数字，match_numbers 应该能够提取
- 包含数字的文本: 答案包含数字但也包含其他文本，可能需要调整提取逻辑
- 纯文本: 答案不包含数字，这类问题可能不是数学问题
- 无answer标签: 转换后仍然没有 answer 标签


## ⚠️ 重要发现：答案准确率验证

虽然数字提取率是100%，但"纯数字答案 0%"意味着所有答案都包含文本。这可能导致以下问题：

1. **多个数字问题**：如果答案文本包含多个数字，`match_numbers`可能提取到错误的数字
2. **文本答案问题**：如果答案是"Yes/No"或选项标签（如"A"），数字提取可能不准确
3. **答案位置问题**：如果答案文本是"The probability is 0.5"，提取的"0.5"可能是正确的，但也可能是reasoning部分中的其他数字

让我们详细检查一下实际答案的格式和提取的数字是否准确。

In [36]:
# ============================================================================
# 详细分析：检查答案内容和提取的数字
# ============================================================================

print("=" * 80)
print("答案内容和提取数字的详细分析")
print("=" * 80)

# 提取answer标签内的完整内容进行分析
answer_pattern = re.compile(
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}",
    flags=re.DOTALL
)

answer_analysis = []

for item in data[:20]:  # 分析前20个样本
    converted = item["converted"]
    
    # 提取answer标签内的完整内容
    answer_match = answer_pattern.search(converted)
    if not answer_match:
        continue
    
    answer_content = answer_match.group(1).strip()
    
    # 提取数字
    number_match = match_numbers.search(converted)
    extracted_number = number_match.group(1) if number_match else None
    
    # 分析答案中的所有数字
    all_numbers_in_answer = re.findall(r'[\d\.]+', answer_content)
    
    # 判断提取的数字是否在答案中，以及是否是第一个/最后一个数字
    number_position = "不在答案中"
    if extracted_number:
        if extracted_number in answer_content:
            # 找到该数字在答案中的位置
            numbers_in_answer = re.findall(r'[\d\.]+', answer_content)
            if numbers_in_answer:
                if extracted_number == numbers_in_answer[0]:
                    number_position = "答案中第一个数字"
                elif extracted_number == numbers_in_answer[-1]:
                    number_position = "答案中最后一个数字"
                elif extracted_number in numbers_in_answer:
                    number_position = f"答案中第{numbers_in_answer.index(extracted_number)+1}个数字"
                else:
                    # 可能是部分匹配
                    number_position = "可能是部分匹配"
            else:
                number_position = "答案中无数字（但能提取到数字）"
    
    answer_analysis.append({
        "index": item["index"],
        "answer_content": answer_content[:150],  # 前150字符
        "extracted_number": extracted_number,
        "all_numbers_in_answer": all_numbers_in_answer,
        "number_position": number_position,
        "answer_length": len(answer_content)
    })

# 显示分析结果
for i, analysis in enumerate(answer_analysis[:10], 1):
    print(f"\n示例 {i} (索引: {analysis['index']}):")
    print("-" * 80)
    print(f"答案内容: {analysis['answer_content']}")
    print(f"提取的数字: {analysis['extracted_number']}")
    print(f"答案中所有数字: {analysis['all_numbers_in_answer']}")
    print(f"数字位置: {analysis['number_position']}")
    print(f"答案长度: {analysis['answer_length']} 字符")

答案内容和提取数字的详细分析

示例 1 (索引: 0):
--------------------------------------------------------------------------------
答案内容: To determine the value of \([(1 \otimes 2) \otimes 3] - [1 \otimes (2 \otimes 3)]\) where the operation \(\otimes\) is defined by \(a \otimes b = \fra
提取的数字: 1
答案中所有数字: ['1', '2', '3', '1', '2', '3', '2', '1', '2', '1', '2', '1', '2', '2', '1', '2', '1', '2', '3', '1', '2', '3', '1', '2', '2', '3', '1', '4', '3', '1', '12', '2', '3', '2', '3', '2', '2', '3', '4', '3', '1', '2', '3', '1', '4', '3', '1', '2', '4', '3', '1', '4', '3', '3', '4', '1', '12', '3', '4', '1', '12', '9', '12', '1', '9', '12', '8', '12', '2', '3', '.']
数字位置: 答案中第一个数字
答案长度: 897 字符

示例 2 (索引: 1):
--------------------------------------------------------------------------------
答案内容: Doug constructs a square window using 8 equal-size panes of glass, with a height to width ratio of 5:2 for each pane. The borders around and between t
提取的数字: 8
答案中所有数字: ['8', '5', '2', '.', '2', '.', '.', '1.', '2', '4', '

In [37]:
# ============================================================================
# 统计分析：潜在问题检测
# ============================================================================

print("=" * 80)
print("潜在问题统计分析")
print("=" * 80)

answer_pattern = re.compile(
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}",
    flags=re.DOTALL
)

statistics = {
    "答案包含多个数字": 0,
    "提取的数字不在答案中": 0,
    "提取的是答案中第一个数字": 0,
    "提取的是答案中最后一个数字": 0,
    "答案中没有数字但能提取到": 0,
    "答案长度超过50字符": 0
}

for item in data:
    converted = item["converted"]
    answer_match = answer_pattern.search(converted)
    if not answer_match:
        continue
    
    answer_content = answer_match.group(1).strip()
    
    # 统计答案长度
    if len(answer_content) > 50:
        statistics["答案长度超过50字符"] += 1
    
    # 提取答案中的所有数字
    numbers_in_answer = re.findall(r'[\d\.]+', answer_content)
    
    if len(numbers_in_answer) > 1:
        statistics["答案包含多个数字"] += 1
    
    # 检查提取的数字
    number_match = match_numbers.search(converted)
    if number_match:
        extracted = number_match.group(1)
        
        # 检查提取的数字是否在答案中
        if extracted not in answer_content:
            # 进一步检查是否是部分匹配或格式问题
            if not any(extracted in num or num.startswith(extracted) for num in numbers_in_answer):
                statistics["提取的数字不在答案中"] += 1
        else:
            if numbers_in_answer:
                if extracted == numbers_in_answer[0]:
                    statistics["提取的是答案中第一个数字"] += 1
                elif extracted == numbers_in_answer[-1]:
                    statistics["提取的是答案中最后一个数字"] += 1
    else:
        if numbers_in_answer:
            statistics["答案中没有数字但能提取到"] += 1

total = len(data)
print(f"总样本数: {total}\n")

for key, count in statistics.items():
    percentage = count / total * 100
    print(f"{key}: {count} ({percentage:.2f}%)")

print("\n" + "=" * 80)
print("问题分析:")
print("=" * 80)

if statistics["答案包含多个数字"] > 0:
    print(f"\n⚠️  发现 {statistics['答案包含多个数字']} 个答案包含多个数字 ({statistics['答案包含多个数字']/total*100:.2f}%)")
    print("   这些答案中，提取的数字可能不是正确答案。")
    print("   建议：检查这些样本，确认提取的数字是否对应正确答案")

if statistics["提取的数字不在答案中"] > 0:
    print(f"\n❌ 发现 {statistics['提取的数字不在答案中']} 个样本提取的数字不在答案中 ({statistics['提取的数字不在答案中']/total*100:.2f}%)")
    print("   这是一个严重问题：提取的数字可能来自reasoning部分，而不是答案部分")
    print("   建议：检查match_numbers正则表达式，确保只从<answer>标签内提取")

if statistics["提取的是答案中最后一个数字"] > 0:
    print(f"\n💡 发现 {statistics['提取的是答案中最后一个数字']} 个样本提取的是最后一个数字 ({statistics['提取的是答案中最后一个数字']/total*100:.2f}%)")
    print("   这可能表示正确答案在答案末尾（如'The answer is 42'）")
    print("   这是合理的，但如果第一个数字才是答案，可能会有问题")

if statistics["答案长度超过50字符"] > 0:
    print(f"\n📝 发现 {statistics['答案长度超过50字符']} 个答案长度超过50字符 ({statistics['答案长度超过50字符']/total*100:.2f}%)")
    print("   这些长答案可能包含解释性文本，需要确保提取的数字是正确答案")

潜在问题统计分析
总样本数: 500

答案包含多个数字: 499 (99.80%)
提取的数字不在答案中: 0 (0.00%)
提取的是答案中第一个数字: 500 (100.00%)
提取的是答案中最后一个数字: 0 (0.00%)
答案中没有数字但能提取到: 0 (0.00%)
答案长度超过50字符: 500 (100.00%)

问题分析:

⚠️  发现 499 个答案包含多个数字 (99.80%)
   这些答案中，提取的数字可能不是正确答案。
   建议：检查这些样本，确认提取的数字是否对应正确答案

📝 发现 500 个答案长度超过50字符 (100.00%)
   这些长答案可能包含解释性文本，需要确保提取的数字是正确答案


In [38]:
# ============================================================================
# 结论和建议
# ============================================================================

print("=" * 80)
print("最终结论和建议")
print("=" * 80)

print("\n✅ 好消息：")
print("   1. 格式匹配率：100% - 所有样本都能被正确转换和识别")
print("   2. 数字提取率：100% - 所有样本都能提取到数字")

print("\n⚠️  需要注意的问题：")

# 检查answer_types是否已定义（来自cell 10）
try:
    if answer_types.get("纯数字答案", 1) == 0:  # 默认为1，如果未定义则跳过这个检查
        print("   1. **所有答案都包含文本**（纯数字答案 0%）")
        print("      - 这意味着 match_numbers 提取的是答案文本中的第一个数字")
        print("      - 如果答案包含多个数字，可能提取到错误的数字")
        print("      - 例如：答案'The answer is 42, and the probability is 0.5'中，")
        print("        当前会提取'42'，但如果正确答案是'0.5'就会出错")
except NameError:
    # 如果没有定义answer_types，使用statistics中的信息推断
    if statistics.get("答案包含多个数字", 0) > 0:
        print("   1. **几乎所有答案都包含文本和多个数字**")
        print("      - 这意味着 match_numbers 提取的是答案文本中的第一个数字")
        print("      - 如果答案包含多个数字，可能提取到错误的数字")
        print("      - 例如：答案'The answer is 42, and the probability is 0.5'中，")
        print("        当前会提取'42'，但如果正确答案是'0.5'就会出错")

if statistics.get("答案包含多个数字", 0) > 0:
    print(f"   2. **{statistics['答案包含多个数字']/total*100:.1f}% 的答案包含多个数字**")
    print("      - 需要确认提取的数字是否总是正确答案")
    print("      - 如果Bespoke-Stratos-17k的答案格式固定（如'The answer is X'），")
    print("        那么提取第一个数字通常是正确的")

if statistics.get("提取的数字不在答案中", 0) > 0:
    print(f"   3. **{statistics['提取的数字不在答案中']/total*100:.1f}% 提取的数字不在答案中**")
    print("      - 这是一个严重问题，需要检查match_numbers的正则表达式")

print("\n💡 建议：")
print("   1. **检查训练数据的答案格式**")
print("      - 确认Bespoke-Stratos-17k的答案格式是否固定")
print("      - 如果格式固定（如'<answer>The answer is 42</answer>'），")
print("        当前的提取逻辑应该是正确的")

print("   2. **在GRPO阶段使用GSM8K数据集**")
print("      - GSM8K的答案通常是纯数字，更适合数字提取和准确率计算")
print("      - Bespoke-Stratos-17k主要用于SFT阶段学习格式")

print("   3. **如果要在Bespoke-Stratos-17k上计算准确率**")
print("      - 需要改进数字提取逻辑，可能需要：")
print("        a) 提取答案中的最后一个数字（如果格式是'The answer is X'）")
print("        b) 或者提取答案中特定位置的数字")
print("        c) 或者需要ground truth答案来验证提取是否正确")

print("\n" + "=" * 80)

最终结论和建议

✅ 好消息：
   1. 格式匹配率：100% - 所有样本都能被正确转换和识别
   2. 数字提取率：100% - 所有样本都能提取到数字

⚠️  需要注意的问题：
   2. **99.8% 的答案包含多个数字**
      - 需要确认提取的数字是否总是正确答案
      - 如果Bespoke-Stratos-17k的答案格式固定（如'The answer is X'），
        那么提取第一个数字通常是正确的

💡 建议：
   1. **检查训练数据的答案格式**
      - 确认Bespoke-Stratos-17k的答案格式是否固定
      - 如果格式固定（如'<answer>The answer is 42</answer>'），
        当前的提取逻辑应该是正确的
   2. **在GRPO阶段使用GSM8K数据集**
      - GSM8K的答案通常是纯数字，更适合数字提取和准确率计算
      - Bespoke-Stratos-17k主要用于SFT阶段学习格式
   3. **如果要在Bespoke-Stratos-17k上计算准确率**
      - 需要改进数字提取逻辑，可能需要：
        a) 提取答案中的最后一个数字（如果格式是'The answer is X'）
        b) 或者提取答案中特定位置的数字
        c) 或者需要ground truth答案来验证提取是否正确



## 💭 重要讨论：SFT 和 GRPO 能否使用同一个数据集？

基于验证结果，我们分析一下 Bespoke-Stratos-17k 是否适合同时在 SFT 和 GRPO 阶段使用。

In [ ]:
# ============================================================================
# 分析：SFT vs GRPO 数据集需求对比
# ============================================================================

print("=" * 80)
print("SFT vs GRPO 数据集需求对比分析")
print("=" * 80)

print("\n📚 SFT 阶段的需求：")
print("   ✅ 需要高质量的推理示例（CoT）")
print("   ✅ 需要规范的格式（<reasoning>...</reasoning><answer>...</answer>）")
print("   ✅ 答案可以是文本或数字（重点是学习格式）")
print("   ✅ 答案可以包含解释性文本")
print("   ✅ 不要求答案可自动验证")

print("\n🎯 GRPO 阶段的需求：")
print("   ✅ 需要可验证的答案（能自动判断对错）")
print("   ⚠️  最好使用纯数字答案（便于数值比较）")
print("   ✅ 需要有明确的 Ground Truth")
print("   ✅ 需要能通过规则提取答案（正则表达式等）")
print("   ✅ 答案验证逻辑要简单可靠")

print("\n" + "=" * 80)
print("Bespoke-Stratos-17k 特性分析：")
print("=" * 80)

print("\n✅ 适合 SFT 的原因：")
print("   - 高质量的长思维链示例")
print("   - 规范的格式（经过转换后）")
print("   - 多样的问题类型")
print("   - 完整的推理过程展示")

print("\n❌ 不适合 GRPO 的原因：")
print("   - ❌ 纯数字答案：0% （所有答案都包含文本）")
print("   - ❌ 答案包含多个数字：99.80% （难以确定哪个是正确答案）")
print("   - ❌ 答案长度超过50字符：100% （包含大量解释性文本）")
print("   - ❌ 数字提取可能不准确（提取的是第一个数字，不一定是正确答案）")
print("   - ⚠️  无法可靠地验证答案正确性")

print("\n" + "=" * 80)
print("结论：")
print("=" * 80)

print("\n❌ **不推荐在 GRPO 阶段使用 Bespoke-Stratos-17k**")
print("\n原因：")
print("   1. **答案验证困难**")
print("      - 99.80%的答案包含多个数字，无法确定哪个是正确答案")
print("      - 100%的答案包含文本，需要复杂的解析逻辑")
print("      - 当前 match_numbers 提取的是第一个数字，可能不是正确答案")
print("\n   2. **奖励函数不可靠**")
print("      - GRPO 依赖准确的奖励信号来训练模型")
print("      - 如果答案提取错误，奖励信号也是错误的")
print("      - 这会导致模型学到错误的模式")
print("\n   3. **训练效果差**")
print("      - 错误的奖励信号会导致训练不稳定")
print("      - 模型可能学会优化错误的目标（如提取第一个数字）")
print("      - 无法有效提升答案准确性")

print("\n✅ **推荐方案：**")
print("   1. **SFT 阶段**：使用 Bespoke-Stratos-17k")
print("      - 重点：学习格式和推理模式")
print("      - 不关心答案准确性")
print("      - 500-2000 步即可")
print("\n   2. **GRPO 阶段**：使用 GSM8K 或其他可验证数据集")
print("      - 重点：提升答案准确性")
print("      - 答案必须是可验证的（纯数字最好）")
print("      - 奖励函数能准确判断对错")
print("\n   3. **为什么不同数据集？**")
print("      - SFT：教'格式'（HOW）")
print("      - GRPO：教'准确性'（WHAT）")
print("      - 分工明确，各司其职")
print("\n   4. **如果必须使用同一数据集：**")
print("      - 需要改进答案提取逻辑（如提取最后一个数字）")
print("      - 需要验证提取的数字是否对应正确答案")
print("      - 可能需要人工标注正确答案位置")
print("      - 建议只用于 SFT 阶段")

print("\n" + "=" * 80)

## 🔄 补充讨论：GSM8K 能否用于 SFT 阶段？

现在我们来分析 GSM8K 是否适合在 SFT 阶段使用。

In [ ]:
# ============================================================================
# 分析：GSM8K 用于 SFT 阶段的可行性
# ============================================================================

print("=" * 80)
print("GSM8K 用于 SFT 阶段的可行性分析")
print("=" * 80)

print("\n📋 GSM8K 数据集特性：")
print("   - 数据格式：问题 + 答案（用 #### 分隔）")
print("   - 答案格式：推理过程 #### 最终答案")
print("   - 推理过程：包含步骤式推理")
print("   - 最终答案：通常是纯数字")
print("   - 数据量：约 8,000 个样本")

print("\n✅ GSM8K 可以用于 SFT 的原因：")
print("   1. **有推理过程**")
print("      - GSM8K 的答案包含推理步骤（在 #### 之前）")
print("      - 可以提取并格式化为 <reasoning>...</reasoning>")
print("      - 最终答案（#### 之后）可以格式化为 <answer>...</answer>")
print("\n   2. **格式简单明确**")
print("      - 问题清晰")
print("      - 答案结构固定（推理 #### 答案）")
print("      - 易于转换和验证")
print("\n   3. **代码支持**")
print("      - 项目中已经有 create_sft_dataset() 函数")
print("      - 可以将 GSM8K 转换为 SFT 格式")
print("      - 转换逻辑：")
print("        reasoning_part = answer.split('####')[0]")
print("        answer_part = answer.split('####')[1]")

print("\n⚠️  GSM8K 用于 SFT 的局限性：")
print("   1. **推理过程相对简单**")
print("      - GSM8K 的推理主要是数学步骤")
print("      - 缺少自我反思和验证过程")
print("      - 不如 Bespoke-Stratos-17k 的推理深度")
print("\n   2. **问题类型单一**")
print("      - 主要是数学问题")
print("      - 缺少其他领域的推理示例")
print("      - 可能学到的推理模式较窄")
print("\n   3. **推理质量有限**")
print("      - GSM8K 的推理步骤相对简短")
print("      - 不如 DeepSeek-R1 蒸馏的推理详细")
print("      - 可能无法很好地学习复杂的推理模式")

print("\n" + "=" * 80)
print("对比分析：GSM8K vs Bespoke-Stratos-17k")
print("=" * 80)

comparison = {
    "维度": ["推理深度", "推理质量", "问题多样性", "格式学习", "适用阶段"],
    "GSM8K": ["简单步骤", "中等（基础数学）", "单一（数学）", "✅ 可以", "SFT（可用但不推荐）"],
    "Bespoke-Stratos-17k": ["深度推理", "高（R1蒸馏）", "多样（多领域）", "✅ 优秀", "SFT（推荐）"]
}

print("\n| 维度 | GSM8K | Bespoke-Stratos-17k |")
print("|------|-------|---------------------|")
for i in range(len(comparison["维度"])):
    dim = comparison["维度"][i]
    gsm8k = comparison["GSM8K"][i]
    bespoke = comparison["Bespoke-Stratos-17k"][i]
    print(f"| {dim} | {gsm8k} | {bespoke} |")

print("\n" + "=" * 80)
print("结论和建议：")
print("=" * 80)

print("\n✅ **GSM8K 可以用在 SFT 阶段，但不推荐作为首选**")
print("\n原因：")
print("   1. **功能上可行**")
print("      - GSM8K 包含推理过程，可以转换为 SFT 格式")
print("      - 项目代码中已有 create_sft_dataset() 函数支持")
print("      - 可以学习基本的格式（<reasoning>...</reasoning><answer>...</answer>）")
print("\n   2. **效果上有限**")
print("      - 推理过程相对简单，不如专门的数据集")
print("      - 问题类型单一，可能学到较窄的推理模式")
print("      - 推理质量不如 Bespoke-Stratos-17k（R1 蒸馏）")

print("\n💡 推荐方案：")
print("   1. **首选：Bespoke-Stratos-17k（SFT 阶段）**")
print("      - 高质量的长推理链")
print("      - 多领域问题")
print("      - DeepSeek-R1 蒸馏的高质量推理")
print("\n   2. **备选：GSM8K（SFT 阶段）**")
print("      - 如果无法获得 Bespoke-Stratos-17k")
print("      - 如果只想快速测试格式学习")
print("      - 如果目标领域就是数学推理")
print("\n   3. **混合使用（可选）**")
print("      - 使用 Bespoke-Stratos-17k 作为主数据集")
print("      - 混入少量 GSM8K（5%-10%）增强数学推理")
print("      - 但要注意两者推理风格可能不一致")

print("\n📝 使用 GSM8K 进行 SFT 的注意事项：")
print("   1. **数据预处理**")
print("      - 需要正确分割推理部分和答案部分（使用 #### 分隔符）")
print("      - 确保推理过程完整（不要截断）")
print("      - 确保答案格式正确（纯数字或简单表达式）")
print("\n   2. **训练策略**")
print("      - 可能需要更多的训练步数（因为推理示例质量较低）")
print("      - 注意监控格式学习效果")
print("      - 如果格式学习不理想，考虑切换到 Bespoke-Stratos-17k")
print("\n   3. **评估验证**")
print("      - 验证模型是否学会了正确的格式")
print("      - 检查推理过程的质量")
print("      - 确保不会在 GRPO 阶段出现格式问题")

print("\n" + "=" * 80)

In [39]:
# ============================================================================
# 总结和建议
# ============================================================================

print("=" * 80)
print("验证总结")
print("=" * 80)

print(f"\n1. 格式匹配率: {results['format_match_rate']:.2f}%")
if results['format_match_rate'] >= 90:
    print("   ✅ 格式转换逻辑工作良好，大部分样本都能正确转换")
else:
    print("   ⚠️  建议检查数据转换逻辑，确保所有格式的标签都能正确转换")

print(f"\n2. 数字提取率: {results['number_extraction_rate']:.2f}%")
if results['number_extraction_rate'] >= 70:
    print("   ✅ 数字提取率良好，pattern 能够识别大部分数值答案")
elif results['number_extraction_rate'] >= 50:
    print("   ⚠️  数字提取率一般，这可能是因为:")
    print("      - 部分答案不是纯数字格式")
    print("      - 答案可能包含单位或其他文本")
    print("   ℹ️  如果这是正常的（非数学问题），则无需修改")
else:
    print("   ❌ 数字提取率较低，建议:")
    print("      - 检查 match_numbers 正则表达式是否合适")
    print("      - 考虑处理包含单位的答案（如 '42 dollars' -> '42'）")

print(f"\n3. 数据集特性:")
print(f"   - 纯数字答案: {answer_types['纯数字']} ({answer_types['纯数字']/total*100:.2f}%)")
print(f"   - 包含数字的文本答案: {answer_types['包含数字的文本']} ({answer_types['包含数字的文本']/total*100:.2f}%)")
print(f"   - 纯文本答案: {answer_types['纯文本']} ({answer_types['纯文本']/total*100:.2f}%)")

print("\n4. 结论:")
if results['format_match_rate'] >= 90:
    print("   ✅ Bespoke-Stratos-17k 数据库的答案可以被现有的 pattern 提取脚本识别")
    print("   ✅ 数据转换逻辑工作正常，可以用于训练")
else:
    print("   ⚠️  部分样本的格式识别存在问题，建议:")
    print("      - 检查失败示例，了解原因")
    print("      - 可能需要调整数据转换或 pattern 提取逻辑")

print("\n" + "=" * 80)

验证总结

1. 格式匹配率: 100.00%
   ✅ 格式转换逻辑工作良好，大部分样本都能正确转换

2. 数字提取率: 100.00%
   ✅ 数字提取率良好，pattern 能够识别大部分数值答案

3. 数据集特性:
   - 纯数字答案: 0 (0.00%)
   - 包含数字的文本答案: 496 (99.20%)
   - 纯文本答案: 4 (0.80%)

4. 结论:
   ✅ Bespoke-Stratos-17k 数据库的答案可以被现有的 pattern 提取脚本识别
   ✅ 数据转换逻辑工作正常，可以用于训练

